In [ ]:
import requests

# Parameters for the request
params = {
    'lat': '24.7136',  # Latitude of the farm location
    'lon': '46.6753',  # Longitude of the farm location
    'startDate': '2024-01-01',  # Start date for historical data
    'endDate': '2024-02-01',  # End date for historical data
    'api_key': 'API_KEY'
}

# Make the API request
response = requests.get('https://api.nasa.gov/smap/v1/soil-moisture', params=params)

# Process the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Failed to fetch data")


params = {
    'location': '24.7136,46.6753',  # Coordinates of the farm
    'startDate': '2024-01-01',
    'endDate': '2024-02-01',
    'api_key': 'YOUR_API_KEY'
}

response = requests.get('https://api.nasa.gov/planetary/precipitation', params=params)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Load your time-series data 
data = pd.read_csv('farm_data.csv')

# Assume 'soil_moisture', 'temperature', 'precipitation' are your features
features = data[['soil_moisture', 'temperature', 'precipitation']].values
target = data['soil_moisture_future'].values  # Future soil moisture/Another feature for prediction

# Normalize the data
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# Reshape features for 1D CNN (samples, time_steps, features)
# Let's assume we are using a window of 10 time steps (sequence length)
time_steps = 10
X = []
y = []
for i in range(time_steps, len(features)):
    X.append(features[i-time_steps:i])
    y.append(target[i])
X = np.array(X)
y = np.array(y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X_train and X_test to have shape (samples, time_steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Build the 1D CNN model
model = Sequential()

# Add convolutional layer
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(time_steps, 3)))

# Max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Dense layer with dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer 
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Rescale the predictions back to original scale
predictions_rescaled = scaler.inverse_transform(predictions)
